# Введение в MapReduce модель на Python


In [50]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [51]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [52]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [53]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [54]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [55]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [56]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [57]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [58]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [59]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [60]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [61]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [62]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [63]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [64]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.5979566499496887),
 (1, 1.5979566499496887),
 (2, 1.5979566499496887),
 (3, 1.5979566499496887),
 (4, 1.5979566499496887)]

## Inverted index

In [65]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [66]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [67]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [68]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('it', 18), ('what', 10)]),
 (1, [('is', 18)])]

## TeraSort

In [69]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.019506239414895044),
   (None, 0.04640102790262857),
   (None, 0.06862724425589506),
   (None, 0.07676381460008641),
   (None, 0.10931170026586245),
   (None, 0.18052664168400945),
   (None, 0.183621328528636),
   (None, 0.2545035695176606),
   (None, 0.29002498254661213),
   (None, 0.38287284638514263),
   (None, 0.44851896644553524),
   (None, 0.4546446907860111)]),
 (1,
  [(None, 0.5338658133497935),
   (None, 0.5765237538257066),
   (None, 0.6020464961596902),
   (None, 0.6023962816222201),
   (None, 0.6103317424033754),
   (None, 0.6459848120456214),
   (None, 0.6694881985108815),
   (None, 0.6983646708939697),
   (None, 0.7245617034490068),
   (None, 0.7883261603887762),
   (None, 0.7928999989756602),
   (None, 0.7954718751365663),
   (None, 0.8467235437466821),
   (None, 0.8512947792618758),
   (None, 0.8694068887456817),
   (None, 0.8931424697841236),
   (None, 0.9416853950959194),
   (None, 0.9773099491824061)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [70]:
import random
from itertools import chain

def flatten(iterables):
    return chain.from_iterable(iterables)

def groupbykey(pairs):
    grouped = {}
    for key, value in pairs:
        grouped.setdefault(key, []).append(value)
    return grouped.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    mapped = flatten(MAP(k, v) for k, v in RECORDREADER)
    grouped = groupbykey(mapped)
    return flatten(REDUCE(k, v) for k, v in grouped)

def MAP(key, value):
    yield key, value

def REDUCE(key, values):
    yield max(values)

def RECORDREADER(count):
    return ((None, random.randint(0, 100)) for _ in range(count))

array = list(RECORDREADER(20))
print("Input Data:", [v for _, v in array])

result = list(MapReduce(array, MAP, REDUCE))
print("Max Value:", result[0])

Input Data: [17, 8, 31, 80, 43, 54, 44, 63, 22, 29, 46, 96, 11, 38, 9, 94, 13, 64, 97, 5]
Max Value: 97


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [71]:
def MAP(key, value):
    yield None, value

def REDUCE(key, values):
    total = sum(values)
    count = len(values)
    yield total / count

def RECORDREADER(count):
    return ((None, random.randint(0, 100)) for _ in range(count))

array = list(RECORDREADER(100))
print("Input Data:", [v for _, v in array])

result = list(MapReduce(array, MAP, REDUCE))
print("Average value:", result[0])

Input Data: [62, 31, 56, 32, 53, 60, 91, 73, 10, 17, 5, 44, 78, 78, 54, 80, 23, 62, 68, 41, 63, 65, 72, 90, 80, 18, 1, 36, 56, 26, 38, 16, 58, 37, 84, 78, 49, 96, 25, 35, 99, 100, 6, 23, 86, 32, 86, 29, 87, 72, 91, 91, 40, 9, 28, 48, 42, 33, 38, 65, 97, 59, 4, 19, 98, 52, 34, 95, 12, 84, 52, 43, 50, 37, 32, 42, 26, 56, 65, 41, 92, 10, 61, 49, 20, 48, 39, 91, 52, 48, 55, 62, 90, 23, 48, 95, 43, 57, 93, 6]
Average value: 52.26


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [72]:
import random
from itertools import groupby, chain

def groupByKey_sorted(iterable):
    grouped_data = {}
    for key, value in iterable:
        grouped_data.setdefault(key, []).append(value)
    return grouped_data.items()

def MAP(key, value):
    yield 1, value

def REDUCE(key, values):
    yield sum(values) / len(values)

def RECORDREADER(count):
    return ((1, random.randint(0, 100)) for _ in range(count))

def MapReduce(RECORDREADER, MAP, REDUCE):
    mapped = chain.from_iterable(MAP(k, v) for k, v in RECORDREADER)
    grouped = groupByKey_sorted(mapped)
    return chain.from_iterable(REDUCE(k, v) for k, v in grouped)

array = list(RECORDREADER(100))
print("Input Data:", array)

result = list(MapReduce(array, MAP, REDUCE))
print("Average value:", result[0])


Input Data: [(1, 54), (1, 77), (1, 66), (1, 89), (1, 93), (1, 29), (1, 85), (1, 61), (1, 96), (1, 70), (1, 35), (1, 81), (1, 16), (1, 75), (1, 56), (1, 46), (1, 48), (1, 58), (1, 27), (1, 72), (1, 96), (1, 96), (1, 38), (1, 18), (1, 82), (1, 61), (1, 1), (1, 20), (1, 82), (1, 26), (1, 79), (1, 76), (1, 95), (1, 44), (1, 3), (1, 18), (1, 52), (1, 28), (1, 80), (1, 26), (1, 62), (1, 23), (1, 98), (1, 27), (1, 66), (1, 54), (1, 69), (1, 99), (1, 26), (1, 63), (1, 78), (1, 0), (1, 88), (1, 25), (1, 42), (1, 5), (1, 99), (1, 90), (1, 78), (1, 29), (1, 23), (1, 26), (1, 88), (1, 19), (1, 59), (1, 3), (1, 21), (1, 13), (1, 59), (1, 66), (1, 79), (1, 37), (1, 28), (1, 22), (1, 78), (1, 77), (1, 1), (1, 25), (1, 53), (1, 24), (1, 31), (1, 2), (1, 25), (1, 76), (1, 83), (1, 61), (1, 75), (1, 72), (1, 89), (1, 84), (1, 0), (1, 12), (1, 84), (1, 0), (1, 12), (1, 61), (1, 89), (1, 96), (1, 71), (1, 34)]
Average value: 52.34


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [73]:
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a dog"""

documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def groupbykey_distributed(mapped_data, partition_function):
    partitions = [{} for _ in range(reducers)]
    for partition in mapped_data:
        for key, value in partition:
            bucket = partitions[partition_function(key)]
            bucket.setdefault(key, []).append(value)
    return [(idx, sorted(bucket.items())) for idx, bucket in enumerate(partitions)]

def PARTITIONER(obj):
    return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    mapped_data = map(lambda reader: (kv for record in reader for kv in MAP(*record)), INPUTFORMAT())
    if COMBINER:
        mapped_data = map(lambda partition: (kv for group in groupbykey_distributed(partition, PARTITIONER) for kv in COMBINER(*group[1])), mapped_data)
    reduced_data = groupbykey_distributed(mapped_data, PARTITIONER)
    results = [(pid, (res for group in partition for res in REDUCE(*group))) for pid, partition in reduced_data]
    total_kv_pairs = sum(len(values) for _, partition in reduced_data for _, values in partition)
    print(f"{total_kv_pairs} key-value pairs were transferred.")
    return results

def INPUTFORMAT():
    split_size = int(np.ceil(len(documents) / maps))
    for i in range(0, len(documents), split_size):
        yield ((f"{docid}:{lineid}", line) for docid, doc in enumerate(documents[i:i + split_size]) for lineid, line in enumerate(doc.split('\n')))

def MAP(docId, line):
    for word in line.split():
        yield word, word

def REDUCE(key, values):
    yield key

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE)
partitioned_output = [(pid, list(data)) for pid, data in partitioned_output]
partitioned_output


50 key-value pairs were transferred.


[(0, ['a', 'dog', 'it', 'what']), (1, ['is'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [74]:
import random

def MAP(elements):
    return [(item, [item]) for item in elements if FILTER_CONDITION(item)]

def REDUCE(mapped_data):
    return [value for group in mapped_data for value in group]

def FILTER_CONDITION(element):
    return element[0] % 2 == 0

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

data = RECORDREADER(5)
print("Input data:", data)

chunk_size = 5
data_chunks = [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]
print("Partitioned data:", data_chunks)

mapped_output = list(map(MAP, data_chunks))
final_output = REDUCE(mapped_output)

print("Filtered result:", final_output)


Input data: [(41, 41, 2), (43, 3, 68), (40, 21, 49), (56, 64, 85), (0, 49, 27)]
Partitioned data: [[(41, 41, 2), (43, 3, 68), (40, 21, 49), (56, 64, 85), (0, 49, 27)]]
Filtered result: [((40, 21, 49), [(40, 21, 49)]), ((56, 64, 85), [(56, 64, 85)]), ((0, 49, 27), [(0, 49, 27)])]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [75]:
import random
from typing import Iterator, NamedTuple

FILTER_VALUES = {25, 90, 69}

def MAP(data):
    selected = tuple(item for item in data if item in FILTER_VALUES)
    return selected, selected

def REDUCE(key, values: Iterator[NamedTuple]):
    return key, key

def RECORDREADER(size):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(size)]

def group_by_key(pairs):
    grouped_data = {}
    for key, value in pairs:
        grouped_data.setdefault(key, []).append(value)
    return grouped_data.items()

data_records = RECORDREADER(5)
print("Input records:", data_records)

mapped_results = list(map(MAP, RECORDREADER(100)))
print("MAP output:", mapped_results)

grouped_results = group_by_key(map(MAP, RECORDREADER(100)))

final_output = [REDUCE(k, v)[0] for k, v in grouped_results]
print("Reduce output:", final_output)


Input records: [(6, 40, 65), (34, 38, 100), (59, 42, 93), (92, 59, 26), (100, 18, 96)]
MAP output: [((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((90,), (90,)), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((90,), (90,)), ((), ()), ((), ()), ((), ()), ((90,), (90,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((25,), (25,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((90,), (90,)), ((), ()), ((25,), (25,)), ((), ()), ((25,), (25,)), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), 

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [76]:
import random
from collections import defaultdict

def MAP(item):
    return item, item

def REDUCE(key, values):
    return key, key

def RECORDREADER(size):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(size)]

def group_by_key(pairs):
    grouped_data = defaultdict(list)
    for key, value in pairs:
        grouped_data[key].append(value)
    return grouped_data.items()

records = RECORDREADER(100)
print("Input records:", records)

mapped_data = map(MAP, records)
print("MAP output:", list(mapped_data))

grouped_data = group_by_key(map(MAP, records))
print("Shuffle output:", list(grouped_data))

final_output = [REDUCE(k, v) for k, v in grouped_data]
print("Reduce output:", final_output)

Input records: [(52, 54, 66), (96, 85, 66), (45, 36, 78), (19, 66, 49), (96, 56, 40), (40, 10, 20), (96, 11, 11), (82, 26, 55), (44, 46, 40), (34, 14, 52), (77, 97, 21), (72, 86, 52), (33, 45, 40), (36, 19, 93), (58, 89, 61), (25, 10, 59), (7, 73, 28), (33, 80, 37), (88, 69, 33), (92, 5, 46), (29, 82, 50), (11, 89, 79), (88, 46, 85), (46, 97, 25), (39, 41, 7), (34, 33, 80), (39, 23, 1), (38, 40, 42), (7, 94, 12), (89, 29, 99), (5, 17, 55), (25, 1, 53), (43, 67, 38), (17, 8, 20), (8, 32, 98), (91, 46, 52), (74, 62, 38), (34, 39, 37), (62, 46, 71), (91, 42, 47), (18, 9, 44), (92, 43, 87), (34, 26, 15), (74, 66, 79), (36, 37, 40), (88, 84, 66), (97, 52, 1), (82, 55, 69), (15, 76, 20), (36, 40, 80), (23, 17, 76), (27, 33, 85), (4, 1, 98), (45, 34, 44), (65, 50, 42), (99, 2, 67), (77, 87, 6), (5, 22, 35), (66, 62, 68), (22, 49, 30), (71, 29, 95), (39, 74, 62), (81, 16, 95), (10, 46, 17), (61, 75, 31), (100, 66, 38), (1, 98, 70), (48, 6, 65), (51, 97, 2), (70, 72, 28), (21, 54, 41), (59, 89,

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [77]:
import random

def MAP(item):
    return item, item

def REDUCE(key, values):
    return (key, key) if len(values) == 2 else None

def RECORDREADER(size):
    return [(random.randint(0, 3), random.randint(0, 3)) for _ in range(size)]

def group_by_key(data):
    grouped = {}
    for key, value in data:
        grouped.setdefault(key, []).append(value)
    return grouped.items()

records = RECORDREADER(100)
print("Input records:", records)

mapped_data = list(map(MAP, records))
print("MAP output:", mapped_data)

grouped_data = list(group_by_key(mapped_data))
print("Shuffle output:", grouped_data)

final_output = list(filter(None, map(lambda pair: REDUCE(*pair), grouped_data)))
print("Reduce output:", final_output)


Input records: [(1, 3), (0, 3), (2, 1), (1, 0), (2, 0), (2, 2), (3, 3), (0, 0), (1, 1), (1, 3), (0, 2), (1, 2), (2, 1), (2, 3), (2, 0), (3, 0), (0, 0), (0, 2), (0, 1), (2, 0), (3, 1), (0, 0), (0, 0), (2, 3), (1, 3), (3, 0), (1, 0), (2, 3), (0, 0), (1, 0), (1, 2), (1, 2), (0, 3), (0, 2), (3, 3), (0, 2), (1, 0), (3, 2), (0, 0), (2, 3), (0, 3), (3, 2), (2, 1), (3, 1), (1, 1), (1, 2), (1, 3), (1, 1), (3, 3), (1, 2), (0, 0), (0, 2), (2, 1), (3, 3), (1, 1), (0, 3), (1, 1), (3, 3), (0, 3), (0, 1), (3, 0), (2, 3), (2, 1), (1, 3), (2, 1), (1, 3), (2, 3), (3, 1), (1, 0), (0, 0), (3, 1), (3, 2), (3, 1), (2, 2), (1, 2), (1, 0), (3, 1), (0, 1), (1, 0), (3, 3), (1, 1), (1, 0), (0, 1), (3, 0), (1, 2), (2, 1), (3, 3), (1, 2), (3, 3), (0, 0), (0, 2), (3, 3), (3, 3), (2, 1), (3, 1), (0, 3), (2, 1), (3, 0), (0, 2), (2, 2)]
MAP output: [((1, 3), (1, 3)), ((0, 3), (0, 3)), ((2, 1), (2, 1)), ((1, 0), (1, 0)), ((2, 0), (2, 0)), ((2, 2), (2, 2)), ((3, 3), (3, 3)), ((0, 0), (0, 0)), ((1, 1), (1, 1)), ((1, 3), 

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [78]:
import random

RELATIONS = [1, 2]

class DataTuple:
    def __init__(self, values: tuple, relation_id: int):
        self.values = values
        self.relation_id = relation_id

def generate_random_tuple(size):
    values = tuple((random.randint(0, 3), random.randint(0, 3)) for _ in range(size))
    relation_id = random.choice(RELATIONS)
    return DataTuple(values, relation_id)

def load_data(count):
    return [generate_random_tuple(3) for _ in range(count)]

def map_function(data_tuple: DataTuple):
    return data_tuple.values, data_tuple.relation_id

def reduce_function(key, values):
    if RELATIONS[0] in values:
        return key, key
    return None

def group_by(iterable):
    grouped = {}
    for key, value in iterable:
        grouped[key] = grouped.get(key, []) + [value]
    return grouped.items()

data_records = load_data(100)

mapped_data = list(map(map_function, data_records))
print("Mapped data:", mapped_data)

grouped_data = list(group_by(mapped_data))
print("Grouped data:", grouped_data)

reduced_data = list(filter(None, map(lambda x: reduce_function(*x), grouped_data)))
print("Reduced data:", reduced_data)


Mapped data: [(((0, 2), (0, 1), (0, 1)), 1), (((1, 3), (1, 0), (3, 3)), 1), (((2, 3), (2, 2), (1, 0)), 2), (((2, 0), (2, 1), (2, 1)), 1), (((0, 1), (2, 1), (2, 3)), 1), (((1, 3), (1, 3), (1, 0)), 1), (((1, 2), (1, 3), (2, 1)), 1), (((0, 0), (2, 1), (2, 0)), 1), (((2, 2), (1, 2), (1, 3)), 1), (((1, 3), (3, 2), (2, 0)), 2), (((2, 1), (1, 0), (3, 0)), 1), (((3, 1), (0, 0), (3, 1)), 1), (((2, 1), (0, 2), (0, 0)), 1), (((1, 2), (1, 1), (2, 0)), 1), (((1, 0), (0, 0), (2, 2)), 2), (((3, 0), (2, 0), (0, 0)), 2), (((0, 2), (0, 3), (0, 1)), 2), (((3, 2), (2, 0), (0, 0)), 2), (((3, 0), (2, 3), (3, 1)), 1), (((0, 1), (0, 1), (0, 1)), 2), (((1, 3), (3, 2), (3, 3)), 2), (((3, 0), (1, 3), (2, 1)), 1), (((3, 2), (3, 0), (1, 2)), 1), (((0, 3), (1, 3), (1, 0)), 2), (((2, 0), (2, 0), (1, 2)), 2), (((0, 0), (2, 2), (3, 0)), 2), (((3, 1), (0, 3), (0, 2)), 2), (((2, 1), (0, 2), (2, 3)), 1), (((2, 3), (1, 1), (3, 2)), 1), (((1, 3), (3, 3), (2, 0)), 1), (((2, 3), (3, 0), (1, 0)), 1), (((0, 1), (2, 0), (3, 0))

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [79]:
import random
from collections import defaultdict

RELATIONS = [1, 2]

class DataTuple:
    def __init__(self, values: tuple, relation_id: int):
        self.values = values
        self.relation_id = relation_id

def generate_random_tuple():
    values = (random.randint(0, 3), random.randint(0, 3))
    relation_id = random.choice(RELATIONS)
    return DataTuple(values, relation_id)

def load_records(count):
    return [generate_random_tuple() for _ in range(count)]

def map_function(data_tuple: DataTuple):
    if data_tuple.relation_id == RELATIONS[0]:
        return (data_tuple.values[1], (data_tuple.relation_id, data_tuple.values[0]))
    return (data_tuple.values[0], (data_tuple.relation_id, data_tuple.values[1]))

def reduce_function(key, values):
    return [(value[0], key, value[1]) for value in values]

def group_by(iterable):
    grouped = defaultdict(list)
    for key, value in iterable:
        grouped[key].append(value)
    return grouped.items()

records = load_records(100)

mapped_data = list(map(map_function, records))
print("Mapped data:", mapped_data)

grouped_data = list(group_by(mapped_data))
print("Grouped data:", grouped_data)

reduced_data = [result for sublist in map(lambda x: reduce_function(*x), grouped_data) for result in sublist]
print("Reduced data:", reduced_data)


Mapped data: [(1, (1, 1)), (3, (1, 3)), (1, (2, 3)), (0, (1, 0)), (3, (2, 2)), (1, (1, 0)), (1, (2, 2)), (2, (2, 1)), (3, (2, 0)), (1, (2, 3)), (1, (2, 0)), (1, (2, 0)), (3, (2, 3)), (0, (1, 1)), (1, (2, 0)), (2, (2, 2)), (2, (1, 2)), (0, (2, 2)), (2, (1, 1)), (1, (2, 0)), (0, (2, 2)), (2, (2, 0)), (3, (2, 2)), (0, (1, 1)), (2, (1, 0)), (1, (2, 3)), (1, (2, 0)), (2, (1, 0)), (0, (1, 1)), (3, (2, 0)), (0, (2, 2)), (2, (2, 2)), (3, (2, 2)), (2, (1, 2)), (1, (1, 0)), (3, (2, 1)), (0, (2, 3)), (2, (2, 2)), (0, (1, 3)), (1, (1, 2)), (1, (1, 1)), (0, (1, 2)), (0, (1, 2)), (0, (2, 2)), (2, (2, 3)), (3, (2, 0)), (3, (1, 3)), (1, (1, 2)), (3, (2, 2)), (2, (1, 3)), (3, (1, 3)), (3, (2, 3)), (2, (1, 2)), (2, (2, 0)), (1, (2, 0)), (0, (1, 0)), (3, (1, 2)), (0, (1, 1)), (1, (1, 0)), (0, (2, 0)), (0, (2, 2)), (0, (1, 0)), (1, (2, 0)), (0, (2, 2)), (1, (2, 2)), (1, (1, 2)), (1, (1, 2)), (0, (2, 3)), (1, (2, 0)), (2, (2, 0)), (1, (1, 3)), (1, (1, 2)), (2, (2, 0)), (2, (1, 3)), (2, (2, 2)), (0, (1, 0))

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [80]:
import random

def generate_random_data():
    return tuple(random.randint(0, 3) for _ in range(3))

def load_data(count):
    return [generate_random_data() for _ in range(count)]

def map_function(data):
    return data[:2]

def reduce_function(key, values):
    return key, sum(values)

def group_by(iterable):
    grouped = {}
    for key, value in iterable:
        grouped[key] = grouped.get(key, []) + [value]
    return grouped.items()

records = load_data(100)

mapped_data = [map_function(record) for record in records]
print("Mapped data:", mapped_data)

grouped_data = list(group_by(mapped_data))
print("Grouped data:", grouped_data)

reduced_data = [reduce_function(key, values) for key, values in grouped_data]
print("Reduced data:", reduced_data)


Mapped data: [(2, 2), (0, 1), (2, 2), (0, 0), (0, 0), (2, 0), (1, 0), (2, 3), (1, 1), (0, 2), (0, 1), (0, 1), (0, 1), (3, 0), (1, 1), (0, 2), (0, 3), (1, 2), (3, 3), (0, 3), (0, 3), (2, 0), (0, 2), (2, 2), (3, 0), (0, 1), (2, 0), (1, 2), (0, 1), (0, 1), (3, 2), (3, 1), (1, 2), (0, 2), (3, 3), (1, 1), (3, 2), (2, 2), (2, 0), (2, 1), (2, 0), (2, 2), (1, 0), (0, 2), (3, 0), (2, 2), (1, 3), (1, 2), (1, 2), (0, 3), (1, 3), (2, 0), (1, 2), (2, 2), (3, 3), (2, 1), (3, 3), (2, 3), (3, 2), (3, 2), (0, 3), (0, 1), (1, 2), (0, 3), (0, 0), (0, 1), (2, 0), (2, 0), (1, 2), (2, 0), (0, 1), (0, 1), (3, 2), (3, 0), (0, 1), (2, 3), (2, 3), (2, 3), (1, 1), (1, 0), (2, 2), (3, 3), (2, 2), (0, 3), (3, 0), (0, 0), (2, 2), (3, 0), (1, 0), (0, 2), (0, 1), (0, 3), (1, 1), (0, 3), (2, 2), (2, 2), (2, 1), (2, 2), (0, 1), (2, 3)]
Grouped data: [(2, [2, 2, 0, 3, 0, 2, 0, 2, 0, 1, 0, 2, 2, 0, 2, 1, 3, 0, 0, 0, 3, 3, 3, 2, 2, 2, 2, 2, 1, 2, 3]), (0, [1, 0, 0, 2, 1, 1, 1, 2, 3, 3, 3, 2, 1, 1, 1, 2, 2, 3, 3, 1, 3, 0, 

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [81]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [82]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
    row_index, col_index = k1
    weight = v1
    for row in range(small_mat.shape[0]):
        yield (row, col_index), weight * small_mat[row][row_index]

def REDUCE(key, values):
    row_index, col_index = key
    total_value = sum(values)
    yield key, total_value


Проверьте своё решение

In [83]:
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [84]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [85]:
import numpy as np

I, J, K = 2, 3, 40
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested):
    for item in nested:
        for sub_item in item:
            yield sub_item

def groupbykey(iterable):
    result = {}
    for key, value in iterable:
        if key not in result:
            result[key] = []
        result[key].append(value)
    return result.items()

def RECORDREADER():
    mat_0 = [((0, i, j), small_mat[i, j]) for i in range(I) for j in range(J)]
    mat_1 = [((1, j, k), big_mat[j, k]) for j in range(J) for k in range(K)]
    yield mat_0
    yield mat_1

def MAP_JOIN(k1, v1):
    mat_num, i, j = k1
    if mat_num == 0:
        yield j, (mat_num, i, v1)
    else:
        yield i, (mat_num, j, v1)

def REDUCE_JOIN(key, values):
    first_mat = [v for v in values if v[0] == 0]
    second_mat = [v for v in values if v[0] == 1]
    for f in first_mat:
        for s in second_mat:
            yield (f[1], s[1]), f[2] * s[2]

def MAP_MUL(k1, v1):
    yield k1, v1

def REDUCE_MUL(key, values):
    yield key, sum(values)

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I_max = max(i for ((i, _), _) in reduce_output) + 1
    K_max = max(k for ((_, k), _) in reduce_output) + 1
    mat = np.zeros((I_max, K_max))
    for (i, k), value in reduce_output:
        mat[i, k] = value
    return mat

def MapReduce(RECORDREADER, MAP, REDUCE):
    mapped = flatten(map(lambda kv: MAP(*kv), flatten(RECORDREADER())))
    grouped = groupbykey(mapped)
    reduced = flatten(map(lambda kv: REDUCE(*kv), grouped))
    return reduced

intermediate = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(lambda: [intermediate], MAP_MUL, REDUCE_MUL)

assert np.allclose(reference_solution, asmatrix(solution)), "Результат неверный!"
print("MapReduce успешно выполнен!")


MapReduce успешно выполнен!


Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [86]:
import numpy as np
from collections import defaultdict

I, J, K = 2, 3, 40

maps = 4
reducers = 2

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested_iterable):
    for iterable in nested_iterable:
        yield from iterable

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
    partitions = defaultdict(dict)
    for map_partition in map_partitions:
        for k2, v2 in map_partition:
            partitions[PARTITIONER(k2)][k2] = partitions[PARTITIONER(k2)].get(k2, []) + [v2]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for partition_id, partition in partitions.items()]

def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

    if COMBINER:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    print(f"{sum(len(vs) for _, vs in flatten([partition for _, partition in reduce_partitions]))} key-value pairs were sent over a network.")
    return reduce_outputs

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I_max = max(i for ((i, _), _) in reduce_output) + 1
    K_max = max(k for ((_, k), _) in reduce_output) + 1
    mat = np.zeros((I_max, K_max))
    for (i, k), value in reduce_output:
        mat[i, k] = value
    return mat

def INPUTFORMAT():
    first_mat = [((0, i, j), val) for i in range(small_mat.shape[0]) for j in range(small_mat.shape[1]) for val in [small_mat[i, j]]]
    second_mat = [((1, j, k), val) for j in range(big_mat.shape[0]) for k in range(big_mat.shape[1]) for val in [big_mat[j, k]]]
    yield first_mat
    yield second_mat

def MAP_JOIN(k1, v1):
    mat_num, i, j = k1
    if mat_num == 0:
        yield j, (mat_num, i, v1)
    else:
        yield i, (mat_num, j, v1)

def REDUCE_JOIN(key, values):
    from_first_mat = [v for v in values if v[0] == 0]
    from_second_mat = [v for v in values if v[0] == 1]
    for f in from_first_mat:
        for s in from_second_mat:
            yield (f[1], s[1]), f[2] * s[2]

def GET_JOINED():
    for j in joined:
        yield j[1]

def MAP_MUL(k1, v1):
    yield k1, v1

def REDUCE_MUL(key, values):
    yield key, sum(values)


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN)
joined = [(partition_id, list(partition)) for partition_id, partition in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL)
pre_result = [(partition_id, list(partition)) for partition_id, partition in mul_output]
print(pre_result)

solution = [v for p in pre_result for v in p[1]]
print(solution)

assert np.allclose(reference_solution, asmatrix(solution)), "Результат неверный!"
print("MapReduce успешно выполнен!")

126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.15802368795787725), ((0, 1), 0.46986821887733315), ((0, 2), 0.5915798489036408), ((0, 3), 0.6937322311686731), ((0, 4), 0.9057670561217299), ((0, 5), 0.32138130881925137), ((0, 6), 0.4025736792525127), ((0, 7), 0.5717409065756092), ((0, 8), 0.4385000615088929), ((0, 9), 0.2827199791131031), ((0, 10), 0.6500223490654395), ((0, 11), 0.420052238257008), ((0, 12), 0.5642753144512662), ((0, 13), 0.05940557251845207), ((0, 14), 0.8068032916256827), ((0, 15), 0.6100716330659384), ((0, 16), 0.6010376359277787), ((0, 17), 0.7835097422056899), ((0, 18), 0.48925540818846874), ((0, 19), 0.17190149439863192), ((0, 20), 0.39672681844832197), ((0, 21), 0.6641484176678698), ((0, 22), 0.2786235725295331), ((0, 23), 0.8769476771157126), ((0, 24), 0.4510895308417871), ((0, 25), 0.25129218736718484), ((0, 26), 0.558093353946624), ((0, 27), 0.578867309949272), ((0, 28), 0.7064780836544501), ((0, 29), 0.4909695065762649), ((0, 30), 0.46265693769

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [87]:
import numpy as np
from collections import defaultdict

I, J, K = 2, 3, 40

maps = 3
reducers = 2

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    grouped = {}
    for key, value in iterable:
        grouped.setdefault(key, []).append(value)
    return grouped.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for key, value in map_partition:
            partition_id = PARTITIONER(key)
            partitions[partition_id].setdefault(key, []).append(value)
    return [(pid, sorted(partition.items())) for pid, partition in enumerate(partitions)]

def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

    if COMBINER is not None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    total_key_value_pairs = sum(len(vs) for _, vs in flatten([partition for _, partition in reduce_partitions]))
    print(f"{total_key_value_pairs} key-value pairs were sent over a network.")

    return reduce_outputs

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    max_i = max(i for ((i, k), _) in reduce_output) + 1
    max_k = max(k for ((i, k), _) in reduce_output) + 1
    mat = np.empty((max_i, max_k))
    for ((i, k), value) in reduce_output:
        mat[i, k] = value
    return mat

def INPUTFORMAT():
    first_mat = [((0, i, j), small_mat[i, j]) for i in range(small_mat.shape[0]) for j in range(small_mat.shape[1])]
    second_mat = [((1, j, k), big_mat[j, k]) for j in range(big_mat.shape[0]) for k in range(big_mat.shape[1])]

    split_size_first = int(np.ceil(len(first_mat) / maps))
    split_size_second = int(np.ceil(len(second_mat) / maps))

    for i in range(0, len(first_mat), split_size_first):
        yield first_mat[i:i + split_size_first]

    for i in range(0, len(second_mat), split_size_second):
        yield second_mat[i:i + split_size_second]

def MAP_JOIN(k1, v1):
    mat_num, i, j = k1
    value = v1
    if mat_num == 0:
        yield (j, (mat_num, i, value))
    else:
        yield (i, (mat_num, j, value))

def REDUCE_JOIN(key, values):
    first_matrix = [v for v in values if v[0] == 0]
    second_matrix = [v for v in values if v[0] == 1]

    for f in first_matrix:
        for s in second_matrix:
            yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():
    for partition in joined:
        yield partition[1]

def MAP_MUL(k1, v1):
    yield (k1, v1)

def REDUCE_MUL(key, values):
    result = sum(values)
    yield (key, result)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN)

joined = [(partition_id, list(partition)) for partition_id, partition in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL)

pre_result = [(partition_id, list(partition)) for partition_id, partition in mul_output]
print(pre_result)

solution = [v for p in pre_result for v in p[1]]
print(solution)

assert np.allclose(reference_solution, asmatrix(solution)), "Результат неверный!"
print("MapReduce успешно выполнен!")


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.6985718947128591), ((0, 1), 0.5187818619668855), ((0, 2), 0.18522576819827932), ((0, 3), 0.5390606131216772), ((0, 4), 0.276389653401727), ((0, 5), 0.21935818243151664), ((0, 6), 0.6534442451056048), ((0, 7), 0.616073558737128), ((0, 8), 0.4492615728609457), ((0, 9), 0.4280040373780428), ((0, 10), 0.10722285666201449), ((0, 11), 0.4364988316758444), ((0, 12), 0.19106004990112194), ((0, 13), 0.5332725432361759), ((0, 14), 0.06840597788092052), ((0, 15), 0.374603710232311), ((0, 16), 0.41206366591568155), ((0, 17), 0.13613580307798306), ((0, 18), 0.42639145496440023), ((0, 19), 0.654861500886909), ((0, 20), 0.6085384382366459), ((0, 21), 0.5322766522486801), ((0, 22), 0.6037083613180272), ((0, 23), 0.3243982460185557), ((0, 24), 0.04407752092191283), ((0, 25), 0.08355597252303754), ((0, 26), 0.036108042346054615), ((0, 27), 0.6009779657282093), ((0, 28), 0.40883086314970635), ((0, 29), 0.4681200099695499), ((0, 30), 0.4855550